In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('training').getOrCreate()

23/07/31 10:27:26 WARN Utils: Your hostname, miguel resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
23/07/31 10:27:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/31 10:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/31 10:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Import machine learning libraries
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Machine learning pipeline
from pyspark.ml import Pipeline

# Import SQL transformer
from pyspark.ml.feature import SQLTransformer, StandardScaler, StringIndexer


from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 0.CARGA DE DATAS

In [3]:
df_train = spark.read.parquet("../outputs/df_train.parquet")
df_test = spark.read.parquet("../outputs/df_test.parquet")

In [4]:
df_train.count()

116217

In [5]:
df_test.count()

25973

In [6]:
objective_name = 'RainTomorrow'

feature_names = ['Temp9am',
                 'RainToday',
                 'WindGustSpeed',
                 'Humidity9am',
                 'WindDir9am',
                 'WindDir3pm',
                 'WindSpeed3pm',
                 'Location',
                 'WindSpeed9am',
                 'Pressure9am',
                 'Rainfall',
                 'Humidity3pm',
                 'WindGustDir',
                 'month',
                 'week_of_year']

Se obtiene el dataset de features en el conjunto de __Entrenamiento__.

In [7]:
data_features = df_train.select(feature_names + [objective_name])

In [8]:
data_features.toPandas()

,Temp9am,RainToday,WindGustSpeed,Humidity9am,WindDir9am,WindDir3pm,WindSpeed3pm,Location,WindSpeed9am,Pressure9am,Rainfall,Humidity3pm,WindGustDir,month,week_of_year,RainTomorrow
0,22.0,No,48.0,61.0,S,SE,26.0,Cairns,22.0,1018.200012,0.0,57.0,S,6,26,0
1,14.4,No,30.0,52.0,NW,N,24.0,GoldCoast,11.0,1023.200012,0.0,40.0,NNE,6,26,0
2,20.4,No,39.0,62.0,undefined,ENE,26.0,Townsville,0.0,1019.900024,0.0,59.0,E,6,26,0
3,13.2,No,72.0,56.0,N,NNW,26.0,Adelaide,24.0,1008.099976,0.0,49.0,NW,6,26,1
4,10.8,No,59.0,70.0,N,N,28.0,MountGambier,31.0,1004.099976,0.0,61.0,N,6,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116212,11.7,No,54.0,50.0,WSW,SSW,37.0,Williamtown,24.0,1023.799988,0.0,34.0,SSW,7,31,1
116213,7.1,Yes,48.0,72.0,SSW,SSW,13.0,Hobart,17.0,1029.599976,2.2,59.0,SW,7,31,1
116214,1.6,No,50.0,97.0,undefined,SE,20.0,Launceston,0.0,1029.000000,0.0,51.0,SE,7,31,0
116215,9.0,No,48.0,54.0,ESE,ESE,24.0,AliceSprings,13.0,1028.500000,0.0,30.0,E,7,31,0


Se obtienen los nombres de variables numéricas y de las categóricas.

In [9]:
float_names = [col_name for col_name, col_type in data_features.dtypes if col_type == 'float']
float_names

['Temp9am',
 'WindGustSpeed',
 'Humidity9am',
 'WindSpeed3pm',
 'WindSpeed9am',
 'Pressure9am',
 'Rainfall',
 'Humidity3pm']

In [10]:
categorical_names = [col_name for col_name, col_type in data_features.dtypes if col_type == 'string']
categorical_names = ['RainToday',
                     'WindDir9am',
                     'WindDir3pm',
                     'Location',
                     'WindGustDir',
                     'month',
                     'week_of_year'
                    ]

In [11]:
df_train = df_train.withColumn('RainTomorrow',
                   col('RainTomorrow').cast(FloatType()))

In [12]:
df_test = df_test.withColumn('RainTomorrow',
                   col('RainTomorrow').cast(FloatType()))

# DESEÑO DE PIPELINE DE PREPROCESAMIENTO

In [13]:
df_train.printSchema()

root
 |-- Temp9am: float (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: float (nullable = true)
 |-- WindGustSpeed: float (nullable = true)
 |-- Humidity9am: float (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed3pm: float (nullable = true)
 |-- Location: string (nullable = true)
 |-- WindSpeed9am: float (nullable = true)
 |-- Pressure9am: float (nullable = true)
 |-- Rainfall: float (nullable = true)
 |-- Humidity3pm: float (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- month: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- week_of_year: string (nullable = true)



In [14]:
# Crear objetos para Imputer
imputer = Imputer(
    inputCols=float_names,
    outputCols=[f"{col}_imputed" for col in float_names]
)

# Crear objetos para StringIndexer
string_indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index")
    for col in categorical_names
]

# Crear objetos para VectorAssembler
numerical_vec_ass = VectorAssembler(
    inputCols=[f"{col}_imputed" for col in float_names],
    outputCol="features_num"
)

categorical_vec_ass = VectorAssembler(
    inputCols=[f"{col}_index" for col in categorical_names],
    outputCol="features_cat"
)

# Crear el objeto VectorAssembler adicional para combinar las columnas numéricas y categóricas
final_vec_ass = VectorAssembler(
    inputCols=["features_num", "features_cat"],
    outputCol="features"
)

In [15]:
# Definir las etapas del Pipeline en el orden deseado, incluyendo el nuevo VectorAssembler adicional
stages = [imputer] + string_indexers + [numerical_vec_ass, categorical_vec_ass, final_vec_ass]

In [16]:
# Crear el Pipeline
pipeline = Pipeline(stages=stages)

In [17]:
# Ajustar el Pipeline al DataFrame df para obtener el modelo de transformación
pipeline_model = pipeline.fit(df_train)

In [18]:
# Utilizar el modelo del Pipeline para transformar df
df_train_transformed = pipeline_model.transform(df_train)

In [19]:
# Muestra el DataFrame resultante con todas las columnas codificadas y combinadas en vectores
# df_train_transformed.toPandas()

# AJUSTE DE REGRESIÓN LOGÍSTICA

In [20]:
# Create a linear regression model
lin_reg = LogisticRegression(featuresCol='features',
                             labelCol='RainTomorrow'
                             )
# Explain parameter 
print(lin_reg.explainParams())
# predictionCol='prediction'

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: RainTomorrow)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimizati

In [21]:
lin_reg_model = lin_reg.fit(df_train_transformed)
df_train_pred = lin_reg_model.transform(df_train_transformed)

In [22]:
reg_eval = BinaryClassificationEvaluator(labelCol="RainTomorrow",
                                         metricName="areaUnderPR")

In [23]:
reg_eval.evaluate(df_train_pred)

0.6651213627151822

In [24]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",
                                              labelCol="RainTomorrow",
                                              metricName="f1")

In [25]:
evaluator.evaluate(df_train_pred)

0.8272277555792967

## CROSS-VALIDACIÓN Y OPTIMIZACIÓN

Define the pipeline

In [26]:
ml_pipeline = Pipeline(stages=[
                               pipeline, # Preprocessing pipeline
                               lin_reg     # Linear regression model
                              ]
                      )

Define the parameter grid. To build a parameter grid, we use the ParamGridBuilder class. We use the original regression object to reference the parameters we want to tune.

In [27]:
param_grid = ParamGridBuilder() \
                               .addGrid(lin_reg.regParam, [0.0, 0.1, 0.3, 0.5]) \
                               .addGrid(lin_reg.elasticNetParam, [0.0, 0.5, 1.0]) \
                               .build()

In [28]:
# reg_eval = BinaryClassificationEvaluator(
#                                          labelCol='RainTomorrow',
#                                           metricName='areaUnderPR' 
#                                         )

In [29]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",
                                              labelCol="RainTomorrow",
                                              metricName="f1")

In [30]:
crossval_ml = CrossValidator(
                             estimator=ml_pipeline, 
                             estimatorParamMaps=param_grid, 
                             evaluator=evaluator, 
                             numFolds=2
                             )

In [31]:
crossval_ml_model = crossval_ml.fit(df_train)

In [32]:
best_model = crossval_ml_model.bestModel
best_score = crossval_ml_model.avgMetrics[0]

print("Best model: ", best_model)
print("Best score: ", best_score)

Best model:  PipelineModel_8b11958e2b81
Best score:  0.826944787593889


In [33]:
best_lin_reg_params = best_model.stages[-1].extractParamMap()

In [34]:
print("Best score (RMSE):", best_score, end="\n\n")
for parameter, value in best_lin_reg_params.items():
    print(f"{str(parameter):50s}, {value}")

Best score (RMSE): 0.826944787593889

LogisticRegression_066e55461d5f__aggregationDepth , 2
LogisticRegression_066e55461d5f__elasticNetParam  , 0.0
LogisticRegression_066e55461d5f__family           , auto
LogisticRegression_066e55461d5f__featuresCol      , features
LogisticRegression_066e55461d5f__fitIntercept     , True
LogisticRegression_066e55461d5f__labelCol         , RainTomorrow
LogisticRegression_066e55461d5f__maxBlockSizeInMB , 0.0
LogisticRegression_066e55461d5f__maxIter          , 100
LogisticRegression_066e55461d5f__predictionCol    , prediction
LogisticRegression_066e55461d5f__probabilityCol   , probability
LogisticRegression_066e55461d5f__rawPredictionCol , rawPrediction
LogisticRegression_066e55461d5f__regParam         , 0.0
LogisticRegression_066e55461d5f__standardization  , True
LogisticRegression_066e55461d5f__threshold        , 0.5
LogisticRegression_066e55461d5f__tol              , 1e-06


# PUEBA EN CONJUNTO DE TESTEO

In [35]:
df_test_pred = best_model.transform(df_test)

In [36]:
# show scores
print(evaluator.evaluate(df_test_pred))

0.8174059583681401
